In [1]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [48]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [49]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')
loansContract = web3.eth.contract(address=loansContractAddress, abi=loansContractABI)

In [50]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0x67d41410a553aa9f4bf77778a6d6ed6e6b02b78b3c96a716272e515979fbe858')

In [51]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0x312b414f110c11084e1282980c67304075f85885f6f197e6f74181eb08a2c1eb')

In [52]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract, _installmentsNum ,  _interest):
    
    transaction = _organizationContract.functions.createLoan(_loanie, _amount, _installmentsNum ,  _interest
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    # _organizationContract.functions.createLoan(_loanie, _loaner, _amount).transact()
    return True

In [99]:
deleteUser()

In [53]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [54]:
# Suppose bank wants to create a loan
#loaner = input("Enter the loaner addrses: ")
loaner = web3.eth.accounts[5]
loaner
### Check to see whether it is a user or organization address
loanerIndex = accountsConract.functions.getIndex(loaner).call()

In [55]:
#loanie = input("Enter the loanie address: ")
loanie = web3.eth.accounts[2]
loanie
loanieIndex = accountsConract.functions.getIndex(loanie).call()

## Get pending loans

In [27]:
# Get pending loans using 3 separate functions
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [37]:
getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)

Enter your password: ········


['349165 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1589886703 14 35 ',
 '896781 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1589886715 7 34 ',
 '246138 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1589886719 7 11 ',
 '212210 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1589886725 10 34 ']

## Confirm or reject loans

In [60]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for id: ' + pendingLoan.split(' ')[0])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 900000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your password: ········
Enter your loanie password: ········
for id: 738786
c/r?c
for id: 554547
c/r?c
for id: 117942
c/r?r
for id: 903660
c/r?r


# Create a new loan

In [59]:
loaner
# Create a new loan
if loanerIndex != -1:
    loanerType = accountsConract.functions.getType(loanerIndex).call()
    if loanerType:
        # We will change this to inputs later
        amount=random.randint(1000,1000000)
        installmentsNum=random.randint(6,15)
        interest=random.randint(5,35)
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=loaner, _amount=amount, _organizationContract=organizationContract, _installmentsNum=installmentsNum, _interest=interest):
            print("Loan created")
else:
    print("This account is not registered in our system.")

Enter the password of the organization: ········
Loan created


## Get user loans

In [61]:
def getLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions. getMyLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [62]:
def getLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    loans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                loans.append(string)
    else:
        print("This account is not registered in our system.")
    return loans

In [63]:
getLoansList(loanieIndex,userContract,accountsConract,loanie)

Enter your password: ········


['738786 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1589891760 9 7 ',
 '554547 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1589891769 7 9 ']

In [64]:
def getInstallments(_userContract, _loanieAddress, _privateKey):
    myId = 1589891760
    transaction = _userContract.functions.getMyInstallments(myId
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getInstallments().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [65]:
def getinstallmentsList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getInstallments(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amount'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [66]:
getinstallmentsList(loanieIndex,userContract,accountsConract,loanie)

Enter your password: ········


['1111 1592483798 0 False ',
 '1111 1595075798 0 False ',
 '1111 1597667798 0 False ',
 '1111 1600259798 0 False ',
 '1111 1602851798 0 False ',
 '1111 1605443798 0 False ',
 '1111 1608035798 0 False ',
 '1111 1610627798 0 False ',
 '1112 1613219798 0 False ']